####  Test hyperopt


### Case.1 : 　Classification Case: Quote from "Data Analysis Technic to win at  Kaggle"

In [31]:
#!pip install hyperopt

In [32]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

train = pd.read_csv('./sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
#test_x = pd.read_csv('./input/sample-data/test_preprocessed.csv')

# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# xgboostによる学習・予測を行うクラス
import xgboost as xgb


class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params.update(self.params)
        num_round = 50
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred


# -----------------------------------
# 探索するパラメータの空間の指定
# -----------------------------------
# hp.choiceでは、複数の選択肢から選ぶ
# hp.uniformでは、下限・上限を指定した一様分布から抽出する。引数は下限・上限
# hp.quniformでは、下限・上限を指定した一様分布のうち一定の間隔ごとの点から抽出する。引数は下限・上限・間隔
# hp.loguniformでは、下限・上限を指定した対数が一様分布に従う分布から抽出する。引数は下限・上限の対数をとった値

from hyperopt import hp

space = {
    'activation': hp.choice('activation', ['prelu', 'relu']),
    'dropout': hp.uniform('dropout', 0, 0.2),
    'units': hp.quniform('units', 32, 256, 32),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.00001), np.log(0.01)),
}

# -----------------------------------
# hyperoptを使ったパラメータ探索
# -----------------------------------
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import log_loss


def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする

    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])

    # Modelクラスを定義しているものとする
    # Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'params: {params}, logloss: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}


# 探索するパラメータの空間を指定する
space = {
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'gamma': hp.quniform('gamma', 0, 0.4, 0.1),
}

# hyperoptによるパラメータ探索の実行
max_evals = 10
trials = Trials()
history = []
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

# 記録した情報からパラメータとスコアを出力する
# （trialsからも情報が取得できるが、パラメータの取得がやや行いづらいため）
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')


[0]	train-error:0.154	eval-error:0.1624                                                                                

[1]	train-error:0.1476	eval-error:0.1604                                                                               

[2]	train-error:0.138533	eval-error:0.1512                                                                             

[3]	train-error:0.1416	eval-error:0.1516                                                                               

[4]	train-error:0.1348	eval-error:0.1456                                                                               

[5]	train-error:0.129867	eval-error:0.1428                                                                             

[6]	train-error:0.1272	eval-error:0.1404                                                                               

[7]	train-error:0.122267	eval-error:0.1424                                                                             

[8]	train-error:0.118933	eval-er

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[16]	train-error:0.094133	eval-error:0.1196                                                                            

[17]	train-error:0.093467	eval-error:0.1192                                                                            

[18]	train-error:0.0908	eval-error:0.1208                                                                              

[19]	train-error:0.088533	eval-error:0.1188                                                                            

[20]	train-error:0.084533	eval-error:0.12                                                                              

[21]	train-error:0.082667	eval-error:0.1196                                                                            

[22]	train-error:0.078667	eval-error:0.1164                                                                            

[23]	train-error:0.077867	eval-error:0.114                                                                             

[24]	train-error:0.0752	eval-err

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[16]	train-error:0.089467	eval-error:0.116

[17]	train-error:0.088533	eval-error:0.112                                                                             

[18]	train-error:0.083733	eval-error:0.112                                                                             

[19]	train-error:0.082933	eval-error:0.1096                                                                            

[20]	train-error:0.0812	eval-error:0.108                                                                               

[21]	train-error:0.081467	eval-error:0.1092                                                                            

[22]	train-error:0.079867	eval-error:0.1108                                                                            

[23]	train-error:0.0764	eval-error:0.1072                                                                              

[24]	train-error:0.074267	eval-error:0.11                                                                    

...

[15]	train-error:0.0772	eval-error:0.116                                                                               

[16]	train-error:0.075067	eval-error:0.1136                                                                            

[17]	train-error:0.073067	eval-error:0.1108                                                                            

[18]	train-error:0.069733	eval-error:0.112                                                                             

[19]	train-error:0.069733	eval-error:0.1092                                                                            

[20]	train-error:0.0672	eval-error:0.11                                                                                

[21]	train-error:0.066267	eval-error:0.1084                                                                            

[22]	train-error:0.0636	eval-error:0.1096                                                                              

[23]	train-error:0.061733	eval-e

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[16]	train-error:0.069733	eval-error:0.11                                                                              

[17]	train-error:0.0692	eval-error:0.1088                                                                              

[18]	train-error:0.0664	eval-error:0.1024                                                                              

[19]	train-error:0.064	eval-error:0.1036                                                                               

[20]	train-error:0.062133	eval-error:0.1028                                                                            

[21]	train-error:0.059733	eval-error:0.1024                                                                            

[22]	train-error:0.055867	eval-error:0.1012                                                                            

[23]	train-error:0.055333	eval-error:0.102                                                                             

[24]	train-error:0.053733	eval-e

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[10]	train-error:0.034	eval-error:0.1148                                                                               

[11]	train-error:0.0304	eval-error:0.1156                                                                              

[12]	train-error:0.026533	eval-error:0.1136                                                                            

[13]	train-error:0.0232	eval-error:0.1144                                                                              

[14]	train-error:0.0204	eval-error:0.1112                                                                              

[15]	train-error:0.0196	eval-error:0.112                                                                               

[16]	train-error:0.017467	eval-error:0.1092                                                                            

[17]	train-error:0.0152	eval-error:0.11                                                                                

[18]	train-error:0.012	eval-erro

### Case2-1 : Regression / predict Tokyo-housing /　target is price:　Modified by Bashii 2020-04-16

In [26]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

train = pd.read_csv("housing_tokyo_chuo_line_week3.csv")
train.head()


,uid,years,minutes,sqrm,distance,renovate,express,price
0,13101-1,34,3,35,2.983801,0,0,24000000
1,13101-2,37,4,20,2.983801,0,0,13000000
2,13101-4,34,3,30,2.983801,0,0,22000000
3,13101-5,35,1,70,2.983801,0,0,69000000
4,13101-6,34,3,25,2.983801,0,0,21000000


In [27]:
train_x = train.iloc[: ,1:-1 ]
train_y = train['price']

In [3]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
#訓練用と検証用のindexを作成する
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# xgboostによる学習・予測を行うクラス
import xgboost as xgb


class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        #　訓練データで学習を行う
        #params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params = {'objective':'reg:squarederror', 'silent': 1 , 'random_state': 71} #回帰の場合
        params.update(self.params) #updateメソッドで辞書型データに追加を行う。
        num_round = 50
        dtrain = xgb.DMatrix(tr_x, label=tr_y) #xgbのデータ型に変換
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred


# -----------------------------------
# 探索するパラメータの空間の指定
# -----------------------------------
# hp.choiceでは、複数の選択肢から選ぶ
# hp.uniformでは、下限・上限を指定した一様分布から抽出する。引数は下限・上限
# hp.quniformでは、下限・上限を指定した一様分布のうち一定の間隔ごとの点から抽出する。引数は下限・上限・間隔
# hp.loguniformでは、下限・上限を指定した対数が一様分布に従う分布から抽出する。引数は下限・上限の対数をとった値

from hyperopt import hp

space = {
    'activation': hp.choice('activation', ['prelu', 'relu']),
    'dropout': hp.uniform('dropout', 0, 0.2),
    'units': hp.quniform('units', 32, 256, 32),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.00001), np.log(0.01)),
}



In [4]:
# -----------------------------------
# hyperoptを使ったパラメータ探索
# -----------------------------------
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error
import math


def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする

    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])

    # Modelクラスを定義しているものとする
    # Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    #score = log_loss(va_y, va_pred) # modified 04-16 
    score = np.sqrt(mean_squared_error(va_y, va_pred)) # score should be rmse 
    print(f'params: {params}, logloss: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}




In [5]:
# 探索するパラメータの空間を指定する
space = {
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'gamma': hp.quniform('gamma', 0, 0.4, 0.1),
}

# hyperoptによるパラメータ探索の実行
max_evals = 10
trials = Trials()
history = []
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

# 記録した情報からパラメータとスコアを出力する
# （trialsからも情報が取得できるが、パラメータの取得がやや行いづらいため）
history = sorted(history, key=lambda tpl: tpl[1])

best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')


[0]	train-rmse:2.91782e+07	eval-rmse:3.07255e+07                                                                       

[1]	train-rmse:2.12728e+07	eval-rmse:2.27529e+07                                                                       

[2]	train-rmse:1.57361e+07	eval-rmse:1.73283e+07                                                                       

[3]	train-rmse:1.18754e+07	eval-rmse:1.36761e+07                                                                       

[4]	train-rmse:9.18095e+06	eval-rmse:1.13002e+07                                                                       

[5]	train-rmse:7.34495e+06	eval-rmse:9.8254e+06                                                                        

[6]	train-rmse:6.06414e+06	eval-rmse:9.03678e+06                                                                       

[7]	train-rmse:5.19228e+06	eval-rmse:8.48597e+06                                                                       

[8]	train-rmse:4.6161e+06	eval-r

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[22]	train-rmse:2.67323e+06	eval-rmse:7.63825e+06                                                                      

[23]	train-rmse:2.57632e+06	eval-rmse:7.64859e+06                                                                      

[24]	train-rmse:2.5228e+06	eval-rmse:7.64583e+06                                                                       

[25]	train-rmse:2.4615e+06	eval-rmse:7.64580e+06                                                                       

[26]	train-rmse:2.42767e+06	eval-rmse:7.65199e+06                                                                      

[27]	train-rmse:2.36227e+06	eval-rmse:7.69225e+06                                                                      

[28]	train-rmse:2.34656e+06	eval-rmse:7.68805e+06                                                                      

[29]	train-rmse:2.30862e+06	eval-rmse:7.69949e+06                                                                      

[30]	train-rmse:2.27861e+06	eval

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[19]	train-rmse:3.7356e+06	eval-rmse:7.40901e+06                                                                       

[20]	train-rmse:3.70422e+06	eval-rmse:7.38875e+06                                                                      

[21]	train-rmse:3.67631e+06	eval-rmse:7.37927e+06                                                                      

[22]	train-rmse:3.66421e+06	eval-rmse:7.36932e+06                                                                      

[23]	train-rmse:3.65626e+06	eval-rmse:7.36877e+06                                                                      

[24]	train-rmse:3.62014e+06	eval-rmse:7.33608e+06                                                                      

[25]	train-rmse:3.5806e+06	eval-rmse:7.33988e+06                                                                       

[26]	train-rmse:3.5428e+06	eval-rmse:7.3522e+06                                                                        

[27]	train-rmse:3.50637e+06	eval

...

[24]	train-rmse:4.27108e+06	eval-rmse:7.30143e+06                                                                      

[25]	train-rmse:4.23253e+06	eval-rmse:7.30505e+06                                                                      

[26]	train-rmse:4.1378e+06	eval-rmse:7.30394e+06                                                                       

[27]	train-rmse:4.08812e+06	eval-rmse:7.29393e+06                                                                      

[28]	train-rmse:4.04125e+06	eval-rmse:7.29904e+06                                                                      

[29]	train-rmse:4.02851e+06	eval-rmse:7.29005e+06                                                                      

[30]	train-rmse:4.01158e+06	eval-rmse:7.29117e+06                                                                      

[31]	train-rmse:3.96994e+06	eval-rmse:7.29881e+06                                                                      

[32]	train-rmse:3.95875e+06	eval

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[25]	train-rmse:3.47214e+06	eval-rmse:7.66963e+06                                                                      

[26]	train-rmse:3.40196e+06	eval-rmse:7.6786e+06                                                                       

[27]	train-rmse:3.37432e+06	eval-rmse:7.67513e+06                                                                      

[28]	train-rmse:3.34679e+06	eval-rmse:7.67822e+06                                                                      

[29]	train-rmse:3.31597e+06	eval-rmse:7.69024e+06                                                                      

[30]	train-rmse:3.28806e+06	eval-rmse:7.69787e+06                                                                      

[31]	train-rmse:3.26304e+06	eval-rmse:7.70426e+06                                                                      

[32]	train-rmse:3.2134e+06	eval-rmse:7.71036e+06                                                                       

[33]	train-rmse:3.1852e+06	eval-

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[24]	train-rmse:3.44941e+06	eval-rmse:7.31615e+06                                                                      

[25]	train-rmse:3.40811e+06	eval-rmse:7.31248e+06                                                                      

[26]	train-rmse:3.36232e+06	eval-rmse:7.29701e+06                                                                      

[27]	train-rmse:3.3395e+06	eval-rmse:7.30968e+06                                                                       

[28]	train-rmse:3.29547e+06	eval-rmse:7.31585e+06                                                                      

[29]	train-rmse:3.27151e+06	eval-rmse:7.29073e+06                                                                      

[30]	train-rmse:3.26108e+06	eval-rmse:7.28131e+06                                                                      

[31]	train-rmse:3.24761e+06	eval-rmse:7.28791e+06                                                                      

[32]	train-rmse:3.19872e+06	eval

In [6]:
print(f'best params:{best[0]}, score:{best[1]:.4f}')

best params:{'gamma': 0.1, 'max_depth': 5, 'min_child_weight': 5.0}, score:7160975.5664


In [7]:
### Best score RMSE  =  7192525.9042

### Case2-2 Fixed parameter xgboost 

In [42]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
#訓練用と検証用のindexを作成する
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

dtrain = xgb.DMatrix(tr_x, label=tr_y) #xgbのデータ型に変換
dvalid = xgb.DMatrix(va_x, label=va_y)


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [43]:
params = {'objective':'reg:squarederror', 'silent':1 , 'random_state':71}
num_round =50
watchlist = [(dtrain,'train'),(dvalid,'eval')]

In [44]:
model = xgb.train(params, dtrain,num_round, evals=watchlist)

[0]	train-rmse:2.92318e+07	eval-rmse:3.07929e+07
[1]	train-rmse:2.14017e+07	eval-rmse:2.2768e+07
[2]	train-rmse:1.59455e+07	eval-rmse:1.74258e+07
[3]	train-rmse:1.21721e+07	eval-rmse:1.36784e+07
[4]	train-rmse:9.5989e+06	eval-rmse:1.12043e+07
[5]	train-rmse:7.94583e+06	eval-rmse:9.78007e+06
[6]	train-rmse:6.86056e+06	eval-rmse:8.97579e+06
[7]	train-rmse:6.11143e+06	eval-rmse:8.4136e+06
[8]	train-rmse:5.66383e+06	eval-rmse:8.11738e+06
[9]	train-rmse:5.37088e+06	eval-rmse:7.91797e+06
[10]	train-rmse:5.16083e+06	eval-rmse:7.74709e+06
[11]	train-rmse:5.02754e+06	eval-rmse:7.69371e+06
[12]	train-rmse:4.93783e+06	eval-rmse:7.67085e+06
[13]	train-rmse:4.7933e+06	eval-rmse:7.65224e+06
[14]	train-rmse:4.70651e+06	eval-rmse:7.57702e+06
[15]	train-rmse:4.60675e+06	eval-rmse:7.55909e+06
[16]	train-rmse:4.53352e+06	eval-rmse:7.55255e+06
[17]	train-rmse:4.46646e+06	eval-rmse:7.53673e+06
[18]	train-rmse:4.41877e+06	eval-rmse:7.54778e+06
[19]	train-rmse:4.37953e+06	eval-rmse:7.51305e+06
[20]	train-rms

In [45]:
### Best score RMSE  =  7330625.00000     ( hyper-opt : 7192525.9042 )

### Case 3-1 : lightgbm -> 0420  'objective': 'regression' works:

In [82]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
#訓練用と検証用のindexを作成する
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# -----------------------------------
# lightgbmの実装
# -----------------------------------
import lightgbm as lgb
from sklearn.metrics import log_loss

# 特徴量と目的変数をlightgbmのデータ構造に変換する
lgb_train = lgb.Dataset(tr_x, tr_y)
lgb_eval = lgb.Dataset(va_x, va_y)

# ハイパーパラメータの設定
params = {'objective': 'regression', 'seed': 71, 'verbose': 0, 'metrics': 'rmse','gamma': 0.1, 'max_depth': 8, 'min_child_weight': 2.0}
# params = {'metric': 'rmse','max_depth' : 9} # Quote from web 
num_round = 100

# 学習の実行
# カテゴリ変数をパラメータで指定している
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
#categorical_features = ['product', 'medical_info_b2', 'medical_info_b3']
model = lgb.train(params, lgb_train, num_boost_round=num_round,
                  #categorical_feature=categorical_features,
                  valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_eval])


#model = lgb.train(params,
#                lgb_train,
#                valid_sets=lgb_eval,
#                num_boost_round=10000,
#                early_stopping_rounds=100,
#                verbose_eval=50)

# バリデーションデータでのスコアの確認
va_pred = model.predict(va_x)
score = np.sqrt(mean_squared_error(va_y, va_pred)) # score should be rmse 
print(f'rmse: {score:.4f}')

# 予測#
#pred = model.predict(test_x)


[1]	train's rmse: 1.98485e+07	valid's rmse: 2.07248e+07
[2]	train's rmse: 1.8256e+07	valid's rmse: 1.91805e+07
[3]	train's rmse: 1.68297e+07	valid's rmse: 1.77515e+07
[4]	train's rmse: 1.557e+07	valid's rmse: 1.65091e+07
[5]	train's rmse: 1.44158e+07	valid's rmse: 1.54033e+07
[6]	train's rmse: 1.34051e+07	valid's rmse: 1.44312e+07
[7]	train's rmse: 1.2508e+07	valid's rmse: 1.35919e+07
[8]	train's rmse: 1.17209e+07	valid's rmse: 1.2819e+07
[9]	train's rmse: 1.10085e+07	valid's rmse: 1.21473e+07
[10]	train's rmse: 1.03881e+07	valid's rmse: 1.15756e+07
[11]	train's rmse: 9.82165e+06	valid's rmse: 1.10552e+07
[12]	train's rmse: 9.32242e+06	valid's rmse: 1.05989e+07
[13]	train's rmse: 8.89234e+06	valid's rmse: 1.01781e+07
[14]	train's rmse: 8.50129e+06	valid's rmse: 9.82563e+06
[15]	train's rmse: 8.13347e+06	valid's rmse: 9.49505e+06
[16]	train's rmse: 7.84217e+06	valid's rmse: 9.23219e+06
[17]	train's rmse: 7.57238e+06	valid's rmse: 8.96509e+06
[18]	train's rmse: 7.3427e+06	valid's rmse: 8

# hyperopt and lightgbm

In [70]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

train = pd.read_csv("housing_tokyo_chuo_line_week3.csv")
train.head()


,uid,years,minutes,sqrm,distance,renovate,express,price
0,13101-1,34,3,35,2.983801,0,0,24000000
1,13101-2,37,4,20,2.983801,0,0,13000000
2,13101-4,34,3,30,2.983801,0,0,22000000
3,13101-5,35,1,70,2.983801,0,0,69000000
4,13101-6,34,3,25,2.983801,0,0,21000000


In [71]:
train_x = train.iloc[: ,1:-1 ]
train_y = train['price']

In [95]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
#訓練用と検証用のindexを作成する
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# lightgbmによる学習・予測を行うクラス
import lightgbm as lgb



class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        #　訓練データで学習を行う
        #params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params = {'objective':'regression', 'silent': 1 , 'random_state': 71} #回帰の場合 lightgbmの場合 object:regression
        params.update(self.params) #updateメソッドで辞書型データに追加を行う。
        num_round = 100
        lgb_train = lgb.Dataset(tr_x, tr_y)
        lgb_eval = lgb.Dataset(va_x, va_y)
        watchlist = [(lgb_train, 'train'), (lgb_eval, 'eval')]
        self.model = lgb.train(params, lgb_train, num_round)
        
        model = lgb.train(params, lgb_train, num_boost_round=num_round,
                  #categorical_feature=categorical_features,
                  valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_eval])

    def predict(self, x):
        #data = lgb.Dataset(x)
        pred = self.model.predict(x)
        return pred


# -----------------------------------
# 探索するパラメータの空間の指定
# -----------------------------------
# hp.choiceでは、複数の選択肢から選ぶ
# hp.uniformでは、下限・上限を指定した一様分布から抽出する。引数は下限・上限
# hp.quniformでは、下限・上限を指定した一様分布のうち一定の間隔ごとの点から抽出する。引数は下限・上限・間隔
# hp.loguniformでは、下限・上限を指定した対数が一様分布に従う分布から抽出する。引数は下限・上限の対数をとった値

from hyperopt import hp

space = {
    'activation': hp.choice('activation', ['prelu', 'relu']),
    'dropout': hp.uniform('dropout', 0, 0.2),
    'units': hp.quniform('units', 32, 256, 32),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.00001), np.log(0.01)),
}



In [96]:
# -----------------------------------
# hyperoptを使ったパラメータ探索
# -----------------------------------
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error
import math


def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする

    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])

    # Modelクラスを定義しているものとする
    # Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    #score = log_loss(va_y, va_pred) # modified 04-16 
    score = np.sqrt(mean_squared_error(va_y, va_pred)) # score should be rmse 
    print(f'params: {params}, logloss: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}




In [97]:
# 探索するパラメータの空間を指定する
space = {
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'gamma': hp.quniform('gamma', 0, 0.4, 0.1),
}

# hyperoptによるパラメータ探索の実行
max_evals = 100
trials = Trials()
history = []
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

# 記録した情報からパラメータとスコアを出力する
# （trialsからも情報が取得できるが、パラメータの取得がやや行いづらいため）
history = sorted(history, key=lambda tpl: tpl[1])

best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')


[1]	train's l2: 3.93962e+14	valid's l2: 4.29517e+14                                                                    
[2]	train's l2: 3.33281e+14	valid's l2: 3.67893e+14                                                                    
[3]	train's l2: 2.8324e+14	valid's l2: 3.15115e+14                                                                     
[4]	train's l2: 2.42424e+14	valid's l2: 2.72551e+14                                                                    
[5]	train's l2: 2.07814e+14	valid's l2: 2.37262e+14                                                                    
  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))



[6]	train's l2: 1.79696e+14	valid's l2: 2.08261e+14                                                                    
[7]	train's l2: 1.5645e+14	valid's l2: 1.84739e+14                                                                     
[8]	train's l2: 1.3738e+14	valid's l2: 1.64327e+14                                                                     
[9]	train's l2: 1.21188e+14	valid's l2: 1.47558e+14                                                                    
[10]	train's l2: 1.07913e+14	valid's l2: 1.33994e+14                                                                   
[11]	train's l2: 9.64648e+13	valid's l2: 1.22217e+14                                                                   
[12]	train's l2: 8.69075e+13	valid's l2: 1.12338e+14                                                                   
[13]	train's l2: 7.90738e+13	valid's l2: 1.03594e+14                                                                   
[14]	train's l2: 7.22719e+13	valid's l2:

[74]	train's l2: 2.44935e+13	valid's l2: 5.02747e+13                                                                   
[75]	train's l2: 2.43604e+13	valid's l2: 5.01354e+13                                                                   
[76]	train's l2: 2.43165e+13	valid's l2: 5.00957e+13                                                                   
[77]	train's l2: 2.41895e+13	valid's l2: 4.99731e+13                                                                   
[78]	train's l2: 2.41098e+13	valid's l2: 4.99604e+13                                                                   
[79]	train's l2: 2.40251e+13	valid's l2: 4.99237e+13                                                                   
[80]	train's l2: 2.39555e+13	valid's l2: 4.99147e+13                                                                   
[81]	train's l2: 2.38807e+13	valid's l2: 4.99093e+13                                                                   
[82]	train's l2: 2.38199e+13	valid's l2:

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))



...

[2]	train's l2: 3.33281e+14	valid's l2: 3.67893e+14                                                                    
[3]	train's l2: 2.8324e+14	valid's l2: 3.15115e+14                                                                     
[4]	train's l2: 2.42424e+14	valid's l2: 2.72551e+14                                                                    
[5]	train's l2: 2.07814e+14	valid's l2: 2.37262e+14                                                                    
[6]	train's l2: 1.79696e+14	valid's l2: 2.08261e+14                                                                    
[7]	train's l2: 1.56443e+14	valid's l2: 1.84852e+14                                                                    
[8]	train's l2: 1.37188e+14	valid's l2: 1.6476e+14                                                                     
[9]	train's l2: 1.20987e+14	valid's l2: 1.47689e+14                                                                    
[10]	train's l2: 1.07425e+14	valid's l2:

[70]	train's l2: 2.46135e+13	valid's l2: 4.97598e+13                                                                   
[71]	train's l2: 2.45795e+13	valid's l2: 4.97123e+13                                                                   
[72]	train's l2: 2.44969e+13	valid's l2: 4.97055e+13                                                                   
[73]	train's l2: 2.43752e+13	valid's l2: 4.95366e+13                                                                   
[74]	train's l2: 2.42777e+13	valid's l2: 4.95146e+13                                                                   
[75]	train's l2: 2.42457e+13	valid's l2: 4.94607e+13                                                                   
[76]	train's l2: 2.40803e+13	valid's l2: 4.95232e+13                                                                   
[77]	train's l2: 2.39787e+13	valid's l2: 4.94987e+13                                                                   
[78]	train's l2: 2.38794e+13	valid's l2:

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))



[11]	train's l2: 9.63032e+13	valid's l2: 1.22587e+14
[12]	train's l2: 8.68002e+13	valid's l2: 1.13485e+14                                                                   
[13]	train's l2: 7.89111e+13	valid's l2: 1.0507e+14                                                                    
[14]	train's l2: 7.22476e+13	valid's l2: 9.82909e+13                                                                   
[15]	train's l2: 6.6758e+13	valid's l2: 9.21301e+13                                                                    
[16]	train's l2: 6.14715e+13	valid's l2: 8.65432e+13                                                                   
[17]	train's l2: 5.72844e+13	valid's l2: 8.23075e+13                                                                   
[18]	train's l2: 5.39442e+13	valid's l2: 7.86275e+13                                                                   
[19]	train's l2: 5.09295e+13	valid's l2: 7.53507e+13                                                       

[79]	train's l2: 2.38479e+13	valid's l2: 4.94878e+13                                                                   
[80]	train's l2: 2.37713e+13	valid's l2: 4.95331e+13                                                                   
[81]	train's l2: 2.36773e+13	valid's l2: 4.94983e+13                                                                   
[82]	train's l2: 2.35891e+13	valid's l2: 4.95288e+13                                                                   
[83]	train's l2: 2.34947e+13	valid's l2: 4.95724e+13                                                                   
[84]	train's l2: 2.33521e+13	valid's l2: 4.97041e+13                                                                   
[85]	train's l2: 2.33314e+13	valid's l2: 4.96896e+13                                                                   
[86]	train's l2: 2.32508e+13	valid's l2: 4.97529e+13                                                                   
[87]	train's l2: 2.31321e+13	valid's l2:

In [92]:
history


[({'gamma': 0.30000000000000004, 'max_depth': 8, 'min_child_weight': 2.0},
  7191881.3101184005),
 ({'gamma': 0.30000000000000004, 'max_depth': 8, 'min_child_weight': 4.0},
  7191881.3101184005),
 ({'gamma': 0.30000000000000004, 'max_depth': 8, 'min_child_weight': 2.0},
  7191881.3101184005),
 ({'gamma': 0.30000000000000004, 'max_depth': 8, 'min_child_weight': 3.0},
  7191881.3101184005),
 ({'gamma': 0.2, 'max_depth': 9, 'min_child_weight': 3.0}, 7205863.630647554),
 ({'gamma': 0.1, 'max_depth': 9, 'min_child_weight': 4.0}, 7205863.630647554),
 ({'gamma': 0.1, 'max_depth': 7, 'min_child_weight': 3.0}, 7280306.869788796),
 ({'gamma': 0.2, 'max_depth': 4, 'min_child_weight': 4.0}, 7527734.653583),
 ({'gamma': 0.4, 'max_depth': 4, 'min_child_weight': 1.0}, 7527734.653583),
 ({'gamma': 0.2, 'max_depth': 3, 'min_child_weight': 4.0}, 7830980.628119422)]

In [93]:
print(f'best params:{best[0]}, score:{best[1]:.4f}')

best params:{'gamma': 0.30000000000000004, 'max_depth': 8, 'min_child_weight': 2.0}, score:7191881.3101


In [94]:
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10027.02trial/s, best loss=?]


{'gamma': 0.30000000000000004, 'max_depth': 8.0, 'min_child_weight': 2.0}